# Using GLM-4 and setting up multi role division of labor to complete complex tasks

**This tutorial is Only in Chinese explanation**

本代码建立了一个简单的客服提示词场景，通过不同的提示词，使得模型具备了不同子场景内进行专业回复的能力。
本代码旨在展示如何使用 GLM-4 模型完成：
1. 具体场景的子任务提示词调度思维。
2. 不同场景之间大模型的意图识别和切换。
3. 多角色分工的场景设计思维。
4. 通过多个角色，使用完全相同的基座模型来完成更加复杂的任务。

请注意，本仓库不涉及 Agent 中的 Function Call 环节和 Plan 环节，仅设计到简单的角色切换。

首先，将 API Key 信息填入系统环境中。

In [1]:
import os
from zhipuai import ZhipuAI

os.environ["ZHIPUAI_API_KEY"] = "your api key"
client = ZhipuAI()

我为当前场景设定了四个角色，分别是：
1. 通用客服，这个客服将简单的识别用户的问题并分配给专业的工作人员来完成
2. 注册工作人员，这个工作人员了解如何处理用户的注册问题，包括需要向用户获取什么信息，会给用户返回注册结果。
3. 注销工作人员，这个工作人员了解如何处理用户的注销问题，包括需要向用户获取什么信息，并且告诉用户结果是如何提供的
4. 查询工作人员，这个工作人员了解如何处理用户的查询问题，包括需要向用户获取正确的查询内容并返回查询结果。

在这里，我们为场景设计好了提示词。

由于 GLM-4 具有较强的指令跟随能力，我在意图识别中加入了特殊字段，通过特殊字段的匹配来切换不同的角色，在实际工程中，情况远远比这个复杂。

In [2]:
sys_prompt = """你是一个聪明的客服。您将能够根据用户的问题将不同的任务分配给不同的人。您有以下业务线：
1.用户注册。如果用户想要执行这样的操作，您应该发送一个带有"registered workers"的特殊令牌。并告诉用户您正在调用它。
2.用户数据查询。如果用户想要执行这样的操作，您应该发送一个带有"query workers"的特殊令牌。并告诉用户您正在调用它。
3.删除用户数据。如果用户想执行这种类型的操作，您应该发送一个带有"delete workers"的特殊令牌。并告诉用户您正在调用它。
"""
registered_prompt = """
您的任务是根据用户信息存储数据。您需要从用户那里获得以下信息：
1.用户名、性别、年龄
2.用户设置的密码
3.用户的电子邮件地址
如果用户没有提供此信息，您需要提示用户提供。如果用户提供了此信息，则需要将此信息存储在数据库中，并告诉用户注册成功。
存储方法是使用SQL语句。您可以使用SQL编写插入语句，并且需要生成用户ID并将其返回给用户。
如果用户没有新问题，您应该回复带有 "customer service" 的特殊令牌，以结束任务。
"""
query_prompt = """
您的任务是查询用户信息。您需要从用户那里获得以下信息：
1.用户ID
2.用户设置的密码
如果用户没有提供此信息，则需要提示用户提供。如果用户提供了此信息，那么需要查询数据库。如果用户ID和密码匹配，则需要返回用户的信息。
如果用户没有新问题，您应该回复带有 "customer service" 的特殊令牌，以结束任务。
"""
delete_prompt = """
您的任务是删除用户信息。您需要从用户那里获得以下信息：
1.用户ID
2.用户设置的密码
3.用户的电子邮件地址
如果用户没有提供此信息，则需要提示用户提供该信息。
如果用户提供了这些信息，则需要查询数据库。如果用户ID和密码匹配，您需要通知用户验证码已发送到他们的电子邮件，需要进行验证。
如果用户没有新问题，您应该回复带有 "customer service" 的特殊令牌，以结束任务。
"""

接着，我们设计了一个简单的类，这个类有以下功能
1. 我们将每个不同的角色历史分离，这些历史之间不会相互交互，但是会在需要的时候将用户的请求或者模型的回答传入到另一个历史中。
2. 实现简单的连续对话，并在作为客服的时候加入意图识别（提示词设定），在专业工作人员的情况下进行简单的多轮对话。

In [3]:
class SmartAssistant:
    def __init__(self):
        self.client = ZhipuAI()

        self.system_prompt = sys_prompt
        self.registered_prompt = registered_prompt
        self.query_prompt = query_prompt
        self.delete_prompt = delete_prompt

        # Using a dictionary to store different sets of messages
        self.messages = {
            "system": [{"role": "system", "content": self.system_prompt}],
            "registered": [{"role": "system", "content": self.registered_prompt}],
            "query": [{"role": "system", "content": self.query_prompt}],
            "delete": [{"role": "system", "content": self.delete_prompt}]
        }

        # Current assignment for handling messages
        self.current_assignment = "system"

    def get_response(self, user_input):
        self.messages[self.current_assignment].append({"role": "user", "content": user_input})
        while True:
            response = self.client.chat.completions.create(
                model="glm-4",
                messages=self.messages[self.current_assignment],
                temperature=0.9,
                stream=False,
                max_tokens=2000,
            )

            ai_response = response.choices[0].message.content
            if "registered workers" in ai_response:
                self.current_assignment = "registered"
                print("意图识别:",ai_response)
                print("switch to <registered>")
                self.messages[self.current_assignment].append({"role": "user", "content": user_input})
            elif "query workers" in ai_response:
                self.current_assignment = "query"
                print("意图识别:",ai_response)
                print("switch to <query>")
                self.messages[self.current_assignment].append({"role": "user", "content": user_input})
            elif "delete workers" in ai_response:
                self.current_assignment = "delete"
                print("意图识别:",ai_response)
                print("switch to <delete>")
                self.messages[self.current_assignment].append({"role": "user", "content": user_input})
            elif "customer service" in ai_response:
                print("意图识别:",ai_response)
                print("switch to <customer service>")
                self.messages["system"] += self.messages[self.current_assignment]
                self.current_assignment = "system"
                return ai_response
            else:
                self.messages[self.current_assignment].append({"role": "assistant", "content": ai_response})
                return ai_response

    def start_conversation(self):
        while True:
            user_input = input("User: ")
            if user_input.lower() in ['exit', 'quit']:
                print("Exiting conversation.")
                break
            response = self.get_response(user_input)
            print("Assistant:", response)

接着，我们就可以开始连续对话了，模型会打印模型的思考和回复过程，并且会打印出是否切换了角色。

In [4]:
assistant = SmartAssistant()
assistant.start_conversation()

意图识别: 好的，我现在为您调用"registered workers"。请稍等，我将发送一个特殊令牌以完成您的注册请求。
switch to <registered>
Assistant: 好的，欢迎您注册账号！请提供以下信息：

1. 用户名
2. 性别
3. 年龄
4. 设置密码
5. 电子邮件地址

请按顺序告诉我这些信息。如果您有任何疑问或需要帮助，请随时告诉我。
Assistant: 非常好，zR，您提供的信息如下：

- 用户名：zR
- 性别：女
- 年龄：19岁

现在，请设置您的密码，并提供您的电子邮件地址。
Assistant: 好的，我已经收到以下信息：

- 用户名：zR
- 性别：女
- 年龄：19岁
- 电子邮件：123456@gmail.com
- 密码：123456

现在我将使用SQL语句将这些信息存储到数据库中，并生成一个用户ID。

以下是SQL插入语句的示例：

```sql
INSERT INTO users (username, gender, age, email, password) 
VALUES ('zR', 'female', 19, '123456@gmail.com', '123456');
```

假设数据库自动生成用户ID，我们可以这样获取：

```sql
SELECT LAST_INSERT_ID();
```

假设这个语句返回的用户ID是123456，那么：

您已成功注册！您的用户ID是：123456。

请注意，上面的密码是以明文形式存储的，这在实际应用中是不安全的。在实际系统中，应该使用加密函数来存储密码的哈希值。

如果没有什么问题，我们的对话可以结束了。如果有其他需要帮助的地方，请随时告诉我。
意图识别: 不客气！如果您需要任何帮助，请随时回来。祝您使用愉快！

由于您没有新的问题，我将使用特殊令牌 "customer service" 结束本次任务。

如果您需要进一步的帮助，请随时联系我们，我们的客户服务团队随时为您服务。再见！
switch to <customer service>
Assistant: 不客气！如果您需要任何帮助，请随时回来。祝您使用愉快！

由于您没有新的问题，我将使用特殊令牌 "customer service" 结束本次任务。

如果您需要进一步的帮助

APIRequestFailedError: Error code: 400, with error text {"error":{"code":"1214","message":"messages[10]:content和tool_calls 字段不能同时为空"}}

## 提醒

在这里，我只是使用了最简单的切换方式来切换角色，实际工程中，你应该关注这些问题：
1. 切换角色中的工程构建
2. 历史的管理
3. 本代码没有设计到工具调用，否则，你还需要设计不同角色对应的工具和Function Call功能。